In [1]:
import malaya
from malaya.preprocessing import Tokenizer
from malaya.text.function import case_of
from malaya.augmentation import (
    replace_similar_consonants, 
    replace_similar_vowels, 
    socialmedia_form,
    vowel_alternate)
from malaya.text import rules
from collections import defaultdict
import random

In [2]:
string = """
Beliau telah dipilih untuk Parlimen Malaysia pada tahun 1976, pada usia 23 tahun, menggantikan bapanya duduk di kerusi Pekan yang berpangkalan di Pahang. Dari tahun 1982 hingga 1986 beliau menjadi Menteri Besar (Ketua Menteri) Pahang, sebelum memasuki persekutuan Kabinet Tun Dr Mahathir Mohamad pada tahun 1986 sebagai Menteri Kebudayaan, Belia dan Sukan. Beliau telah berkhidmat dalam pelbagai jawatan Kabinet sepanjang baki tahun 1980-an dan 1990-an, termasuk sebagai Menteri Pertahanan dan Menteri Pelajaran. Beliau menjadi Timbalan Perdana Menteri pada 7 Januari 2004, berkhidmat di bawah Perdana Menteri Tun Dato' Seri Abdullah Ahmad Badawi, sebelum menggantikan Badawi setahun selepas Barisan Nasional mengalami kerugian besar dalam pilihan raya 2008. Di bawah kepimpinan beliau, Barisan Nasional memenangi pilihan raya 2013, walaupun buat kali pertama dalam sejarah Malaysia pembangkang memenangi majoriti undi popular.
"""

In [3]:
string2 = """
SUNGAI PETANI: Kerjasama rentas parti yang dicadangkan bukan bermaksud membentuk kerajaan perpaduan seperti digembar-gembur sesetengah pihak, tegas Datuk Seri Ismail Sabri Yaakob.
Menurut Perdana Menteri, pemimpin pembangkang, sebaliknya, dipelawa untuk membantu kerajaan dengan menyertai Majlis Pemulihan Negara (MPN) dan Jawatankuasa Khas Menangani COVID-19.
"Tidak ada kerajaan perpaduan yang melibatkan pembangkang dalam Kabinet, tetapi kerjasama itu bukan semestinya pembangkang berada dalam Kabinet.
"Kalau nak bantu tangani COVID-19, kita ada Jawatankuasa Khas Menangani COVID-19, boleh bersama dalam jawatankuasa ini.
"""

In [4]:
splitted = malaya.text.function.split_into_sentences(string)

In [5]:
replace_normalizer = defaultdict(list)
for k, v in rules.rules_normalizer.items():
    if v.count(' ') == 0:
        replace_normalizer[v].append(k)

In [69]:
tokenizer = Tokenizer(duration = False).tokenize

def augment(string):
    
    r = []
    for word in tokenizer(string):
        original_word = word
        word_lower = word.lower()
        try:
            s = socialmedia_form(word_lower)
            if len(s):
                word = case_of(word)(s[0])
            else:
                if word_lower in replace_normalizer and random.random() >= 0.5:
                    word = case_of(word)(random.choice(replace_normalizer[word_lower]))
            
            word = case_of(word)(vowel_alternate(word, 0.9))
            word = case_of(word)(replace_similar_consonants(word, 0.95))
            word = case_of(word)(replace_similar_vowels(word, 0.95))
                
        except Exception as e:
            word = original_word
            pass
        
        r.append(word)
    return ' '.join(r)

In [72]:
augment(splitted[0])

'Beliau tela dipili tk Parlimen Msia pade tahum 1976 , pade usia 23 tahun , ngegantiin bapanye duduk d kerusi Pekan yg berpangkalan di Pahang .'